In [18]:
from mido import MidiFile
import os
from mido import MetaMessage

# directory name should be midis, at the same level as this file. 

In [7]:
filepath = os.path.join(os.getcwd(),"midis", "mozart", "mz_311_1.mid")
mid = MidiFile(filepath, clip=True)
print(mid)

<midi file '/Users/lena/git/NLP_study_buddy/midis/mozart/mz_311_1.mid' type 1, 10 tracks, 7493 messages>


In [11]:
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        if msg.type == 'key_signature':
            print(msg)
            

Track 0: Klaviersonate Nr. 8 KV 311 1. Satz
<meta message key_signature key='D' time=0>
Track 1: Piano right
Track 2: Piano left
Track 3: Pedal
Track 4: Mozart: Sonate KV 311
Track 5: Copyright © 2006 by Bernd Krüger
Track 6: http://www.piano-midi.de
Track 7: Edition: 2013-06-05
Track 8: Spur 8
Track 9: Spur 9


In [19]:
MetaMessage('key_signature', key='C#', mode='major')


ValueError: mode is not a valid argument for this message type